### Setting environment variables

In [1]:
# Import the os module to interact with the operating system environment variables
import os

# Set the LANGCHAIN_TRACING_V2 environment variable to enable tracing for LangChain version 2
#os.environ["LANGCHAIN_TRACING_V2"] = "true"


# Set the LANGCHAIN_PROJECT environment variable to specify the project name in LangChain
#os.environ['LANGCHAIN_PROJECT'] = "agentic-workflow"

# Set the LANGSMITH_USER_HANDLE environment variable with your LangSmith user handle
#os.environ['LANGSMITH_USER_HANDLE'] = "hrisi-learn"

# Set the SERPAPI_API_KEY environment variable with your SERP API key
#os.environ['SERPAPI_API_KEY'] = ""

# Set Azure OpenAI API key
#os.environ["AZURE_OPENAI_API_KEY"] = ""
#os.environ["AZURE_OPENAI_EMBEDDING_MODEL_NAME"] = ""
#os.environ["AZURE_OPENAI_ENDPOINT"] = ""

In [2]:
# Import the Client class from the langsmith module to create a client instance
from langsmith import Client

# Initialize a new client instance using the Client class
client = Client()

### Loading libraries

In [3]:
# Import the itemgetter function from the operator module to extract items from objects
from operator import itemgetter

# Import the hub module from langchain to access its functionalities
from langchain import hub

# Import the AgentExecutor class from langchain.agents to execute agent actions
from langchain.agents import AgentExecutor

# Import the load_tools function from langchain.agents to load necessary tools for agents
from langchain.agents import load_tools

# Import the format_to_openai_functions function from langchain.agents.format_scratchpad
# to convert scratchpad content into OpenAI functions
from langchain.agents.format_scratchpad import format_to_openai_functions

# Import the OpenAIFunctionsAgentOutputParser class from langchain.agents.output_parsers
# to parse the output of agents into OpenAI functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

# Import the ChatOpenAI class from langchain.chat_models to interact with OpenAI chat models
from langchain.chat_models import ChatOpenAI,AzureChatOpenAI

# Import the format_tool_to_openai_function function from langchain.tools.render
# to format tools into OpenAI functions
from langchain.tools.render import format_tool_to_openai_function

In [5]:
!pip install numexpr

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/95.0 kB ? eta -:--:--
   -------------------------------------- - 92.2/95.0 kB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 92.2/95.0 kB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 92.2/95.0 kB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 92.2/95.0 kB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 92.2/95.0 kB 5.1 MB/s eta 0:00:01
   -------------------------------------- - 92.2/95.0 kB 5.1 MB/s eta 0:00:01
   ---------------------------------------- 95.0/95.0 kB 285.2 kB/s eta 0:00:00


### Initializing LLMs and Tools

In [4]:
# Create an instance of the ChatOpenAI class to interact with OpenAI's chat models
#llm = ChatOpenAI()
llm = AzureChatOpenAI(model="hrisikesh-gpt-35-turbo", temperature=0,api_version="2024-02-01")
# Load tools such as serpapi, wikipedia, and llm-math using the load_tools function
tools = load_tools(["serpapi", "wikipedia", "llm-math"], llm=llm)

# Bind the loaded tools to the language model (llm) by converting them into OpenAI functions
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])

c:\program files\python38\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureChatOpenAI`.
  warn_deprecated(
c:\program files\python38\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.2.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


### Prompt design

In [5]:
# Use the hub.pull method from the langchain hub to retrieve a specific prompt
prompt = hub.pull("wfh/agent-lcel-prompt")

In [7]:
# Define a function named format_scratchpad that takes a dictionary as input
# The dictionary is expected to contain a key "intermediate_steps" whose value is processed
# using the format_to_openai_functions function to convert intermediate steps into a string
def format_scratchpad(x: dict) -> str:
    return format_to_openai_functions(x["intermediate_steps"])

# Define a function named get_chat_history that takes a list as input
# The list is expected to contain a dictionary with a key "chat_history"
# If the "chat_history" key exists, its value is returned; otherwise, an empty list is returned
def get_chat_history(x: list) -> str:
    return x.get("chat_history", [])

### Creating agents

In [8]:
# Define an agent configuration dictionary with specific functions mapped to keys
# "input" is mapped to the itemgetter function to extract the "input" field from a given object
# "agent_scratchpad" is mapped to the format_scratchpad function to process intermediate steps
# "chat_history" is mapped to the get_chat_history function to retrieve chat history
agent_config = {
    "input": itemgetter("input"),  # Map "input" to the itemgetter function
    "agent_scratchpad": format_scratchpad,  # Map "agent_scratchpad" to the format_scratchpad function
    "chat_history": get_chat_history  # Map "chat_history" to the get_chat_history function
}

# Apply the agent configuration to a prompt using the pipe operator
# This involves pulling the prompt from the hub, binding it with the loaded tools (llm_with_tools),
# and then processing it through the agent configuration
# The result is then parsed by the OpenAIFunctionsAgentOutputParser to format the output
agent = (
    agent_config
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

In [9]:
# Instantiate an AgentExecutor object with the specified agent configuration and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### Questions to agents

In [10]:
inputs = ["who is Indian prime minister",
          "What is US President's age and return the number age multiplied by two",
          "what is the name of OpenAI's new AI model?"]

In [11]:
results = agent_executor.batch([{"input": x} for x in inputs], return_exceptions=True)



> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...


> Entering new AgentExecutor chain...
I'm sorry, but I cannot answer that question without additional information. Which US President are you referring to?

> Finished chain.


In [11]:
results

[ValueError('An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse tool input: {\'arguments\': "const wikipedia = require(\'wikipedia\');\\n\\nasync function indianPrimeMinister() {\\n  const page = await wikipedia.page(\'Prime Minister of India\');\\n  const summary = await page.summary();\\n  const intro = summary.intro;\\n  const regex = /([A-Z][a-z]+ ){1,2}(Modi|Singh|Gandhi)/;\\n  const match = intro.match(regex);\\n  return match[0];\\n}\\n\\nindianPrimeMinister();", \'name\': \'functions\'} because the `arguments` is not valid JSON.'),
 {'input': "What is US President's age and return the number age multiplied by two",
  'output': "I'm sorry, but I cannot answer that question without additional information. Which US President are you referring to?"},
 ValueError('An output parsing error occurred. In order to pass this error back to the agent